In [ ]:
# The class below has an implementation of a genetic algorithm, to which we can supply any fitness function which we will 
# implement in the assignments
import random
import bisect
from collections import OrderedDict

class GA:
    # A genetic algorithm, keeping track of its current population and its fitness.
    # 
    # Arguments:
    # fitness_function            -> calculates the fitness of an individual
    # generate_random_inidividual -> generate the first generation of individuals for the problem
    # population_size             -> the amount of individuals in a generation
    # parents                     -> the amount of individuals needed to create a child
    # clones                      -> the amount of best performing individuals to keep for the next generation
    def __init__(self, fitness_function, generate_random_individual, create_child, population_size=100, parents=2, clones=1):
        self.fitness = fitness_function
        self.create_child = create_child
        self.PARENTS = parents
        self.CLONES = clones

        self._generate_first_population(population_size, generate_random_individual)
        self._compute_population_performance()

    # Generates a new population and computes its fitness
    def next(self):
        self._generate_next_population()
        self._compute_population_performance()

    # Generate the first population using generate_random_individual
    def _generate_first_population(self, sizePopulation, generate_random_individual):
        # Generate the initial population, using the given the generate_random_individual function
        self.population = []
        for _ in range(sizePopulation):
            self.population.append(generate_random_individual())

    # Compute the fitness of the current population, keeping track of the fittest individual
    def _compute_population_performance(self):
        total_fitness = 0
        population_fitness_array = []
        max_fitness = 0
        for individual in self.population:
            fitness = self.fitness(individual)
            if fitness > max_fitness:
                max_fitness = fitness
                self.fittest_individual = individual
            total_fitness += fitness
            population_fitness_array.append(total_fitness)
        self.population_fitness = population_fitness_array

    # Clone the fittest individuals and breed a new generation using the previous generation.
    def _generate_next_population(self):
        new_population = []
        for _ in range(len(self.population) - self.CLONES):
            new_population.append(self.create_child(self._choose_parents()))
        for _ in range(self.CLONES):
            new_population.append(self.fittest_individual)
        self.population = new_population
        self._compute_population_performance()
        
    # Get an individual by rolling a rad of fortune, each individual with a chance proportionate to its fitness.
    def _wheel_of_fortune(self):
        return self.population[bisect.bisect_left(self.population_fitness, random.random() * self.population_fitness[-1])]

    # Generate an array of random parents for a single child, using the wheel of fortune
    def _choose_parents(self):
        return [self._wheel_of_fortune() for _ in range(self.PARENTS)]

In [ ]:
# Lets try to solve a knapsack problem using the genetic algorithm implementation defined above
knapsack_data = {
    'max_size':100,        # the maximum allowed total size of items in the knapsack
    'max_weight':100,      # the maximum allowed total weight of items in the knapsack
    'items':[[14, 13, 10], # the item descriptions [size, weight, value]
                     [10, 9, 17],[19, 14, 12],[11, 17, 15],[10, 12, 7],[18, 5, 11],[9, 16, 5],[9, 7, 10],[13, 16, 9],[10, 5, 7],[14, 7, 15],[20, 15, 18],[8, 6, 17],[18, 9, 7],[18, 14, 9],[14, 8, 9],[13, 12, 10],[6, 18, 16],[18, 15, 10],[6, 20, 14],[10, 17, 15],[16, 18, 19],[19, 15, 18],[12, 13, 6],[19, 5, 17],[10, 13, 14],[16, 17, 15],[19, 9, 6],[17, 16, 8],[13, 15, 20]]}

MUTATIONS = 2
NUM_ITEMS = len(knapsack_data["items"])
BACKPACK_SIZE, BACKPACK_WEIGHT = knapsack_data["max_size"], knapsack_data["max_weight"]
ITEMS = knapsack_data["items"]

In [ ]:
# First lets think about what our individuals look like. In the best solution we will have a knapsack full of items where
# the sum of the values is high. We can represent this by the ordering the items in a list structure (using the index in item.data). The knapsack will 
# always be filled by starting from the first item in the list, and continuing until the knapsack is full.
#
# So for our problem we can write the following function to generate the random individuals

def generate_random_individual():
    individual = list(range(NUM_ITEMS))
    random.shuffle(individual)
    return individual

# Now we are ready to think about how a new generation will be build up. Let us follow nature approximately and create a
# child from two parents with maybe some small mutations

def create_child(parents):
    child = crossover(parents)
    for _ in range(MUTATIONS):
        mutate(child)
    return child

# ASSIGNMENT 1:
# 
# Implement the mutations function, it should randomly swap two elements items of our indidual (use random_postition)
def mutate(individual):
    # TO REMOVE
    i1, i2 = random_position(), random_position()
    tmp = individual[i1]
    individual[i1] = individual[i2]
    individual[i2] = tmp

def random_position():
    # Pick a random item
    return random.randint(0, NUM_ITEMS - 1)

# Create a new pack/child list by iterating over the parent their pack list from begin to end,
# randomly picking the next parent for every item in the list. It only adds items that are not
# already on the new pack list, and iterates over all parent pack lists, until the list contains all items.
def crossover(parents):
    index = [0 for _ in range(len(parents))]
    child = OrderedDict()
    while len(child) < len(ITEMS):
        gene_parent = random.randint(0, len(parents)-1)
        parent = parents[gene_parent]
        gene = parent[index[gene_parent]]
        index[gene_parent] += 1
        if gene not in child:
            child.update({gene: gene})
    return list(child.keys())

In [ ]:
# TEST ASSIGNMENT 1
#

def test():
    for i in range(10):

        individual = generate_random_individual()
        mutated = individual[:]
        mutate(individual)

        ind = []
        for i in range(NUM_ITEMS):
            if individual[i] != mutated[i]:
                ind.append(i)
        
        if len(ind) == 0 or len(ind) == 2 and individual[ind[0]] == mutated[ind[1]] and individual[ind[1]] == mutated[ind[0]]:
            continue
        else:
            return "FAIL"
    return "PASS"

print(test())

In [ ]:
# Now lets define a fitness function for an individual, remember the restrictions of the knapsack problem allow a maximum
# total weight and size in the backpack. And the goal is to have a high value
#
# ASSIGNMENT 2
#
# implement the get_knapsack_value function that returns the sum of the values that fit in the backpack.
# this will be used as the initial fitness function

BACKPACK_SIZE, BACKPACK_WEIGHT = knapsack_data["max_size"], knapsack_data["max_weight"]
ITEMS = knapsack_data["items"]

def fitness(individual):
    return get_knapsack_value(individual)

def get_knapsack_value(individual):
    size, weight = BACKPACK_SIZE, BACKPACK_WEIGHT
    value = 0
    for item_id in individual:
        item = ITEMS[item_id]

        size -= item[0]
        weight -= item[1]

        if (size >= 0) and (weight >= 0):
            value += item[2]
        else:
            return value

In [ ]:
# TEST ASSIGNMENT 2
#       
if fitness(list(range(30))) == 87 and get_knapsack_value(list(range(30))) == 87:
    print("PASS")
else:
    print("FAIL")

In [ ]:
# Lets run the genetic algorithm and see how it does
#
from string import Template
import datetime
random.seed(1)

ga = GA(fitness, generate_random_individual, create_child)

GENERATION_LOG = Template('[$time] - Generation $generation - Average knapsack value of $average_knapsack_value')
BEST_STATISTICS = Template('Fittest with $fitness knapsack value')
for generation in range(20):
    ga.next()
    knapsack_values = list(map(lambda x: get_knapsack_value(x), ga.population))
    average_knapsack_value = sum(knapsack_values) / float(len(knapsack_values))
    fittest_knapsack_value = get_knapsack_value(ga.fittest_individual)
    print(GENERATION_LOG.substitute(time=datetime.datetime.now(),generation=generation, average_knapsack_value=average_knapsack_value))
    print(BEST_STATISTICS.substitute(fitness=fittest_knapsack_value))

In [ ]:
# A larger dataset
random.seed(1)

knapsack_data = {
    'max_size':200,        # the maximum allowed total size of items in the knapsack
    'max_weight':200,      # the maximum allowed total weight of items in the knapsack
    'items':[[29, 18, 25], # the item descriptions [size, weight, value]
             [10, 26, 21],[10, 29, 26],[24, 11, 16],[12, 19, 25],[20, 19, 12],[14, 11, 12],[17, 24, 28],[21, 19, 26],[12, 18, 26],[27, 30, 18],[23, 26, 20],[26, 30, 18],[19, 13, 30],[27, 11, 30],[19, 23, 17],[18, 27, 29],[27, 21, 15],[22, 14, 19],[14, 26, 22],[24, 10, 28],[20, 29, 12],[11, 14, 15],[17, 24, 15],[29, 24, 14],[26, 16, 23],[27, 15, 29],[11, 27, 18],[11, 10, 28],[25, 22, 14],[11, 16, 26],[14, 23, 26],[23, 18, 17],[13, 27, 28],[18, 21, 15],[23, 14, 10],[18, 25, 29],[27, 20, 21],[23, 12, 21],[20, 18, 26],[11, 23, 10],[13, 23, 29],[23, 16, 13],[16, 28, 18],[15, 20, 13],[12, 27, 22],[24, 22, 15],[30, 24, 14],[19, 16, 30],[22, 18, 21],[11, 28, 30],[28, 24, 17],[18, 29, 22],[23, 11, 10],[17, 11, 24],[14, 27, 18],[11, 18, 14],[12, 29, 26],[17, 24, 23],[17, 25, 14],[10, 12, 15],[14, 23, 25],[12, 25, 15],[11, 18, 26],[29, 24, 21],[29, 20, 24],[11, 10, 24],[24, 12, 17],[13, 26, 22],[17, 12, 15],[10, 24, 30],[10, 25, 14],[25, 18, 26],[23, 21, 19],[25, 28, 21],[10, 22, 11],[11, 23, 29],[13, 12, 29],[30, 25, 26],[18, 15, 23],[10, 25, 16],[14, 19, 10],[17, 12, 24],[27, 19, 29],[28, 22, 18],[10, 23, 30],[24, 21, 13],[20, 23, 24],[28, 17, 15],[12, 21, 11],[12, 10, 17],[23, 13, 15],[13, 22, 27],[11, 18, 15],[18, 28, 17],[23, 20, 28],[28, 30, 27],[23, 13, 12],[13, 23, 14],[12, 21, 24],[10, 15, 30],[25, 30, 13],[19, 23, 11],[26, 11, 16],[15, 30, 16],[14, 21, 16],[18, 16, 29],[10, 30, 16],[19, 28, 10],[15, 17, 12],[18, 28, 13],[26, 24, 19],[16, 14, 25],[15, 11, 10],[24, 18, 13],[27, 14, 15],[16, 15, 21],[14, 26, 19],[11, 21, 22],[25, 15, 23],[12, 27, 29],[19, 22, 24],[23, 29, 18],[26, 13, 24],[10, 14, 29],[28, 21, 29],[22, 24, 28],[20, 18, 15],[26, 28, 22],[29, 18, 12],[13, 26, 28],[26, 21, 12],[28, 17, 26],[23, 14, 21],[10, 29, 12],[18, 26, 15],[15, 22, 23],[22, 29, 17],[17, 10, 26],[12, 19, 26],[27, 15, 12],[12, 11, 26],[14, 29, 28],[26, 11, 13],[29, 14, 19],[20, 26, 25],[29, 15, 12],[14, 11, 12],[21, 19, 22],[27, 17, 24],[14, 15, 30],[23, 14, 21],[21, 28, 12],[10, 19, 20],[28, 10, 29],[26, 21, 24],[18, 26, 11],[27, 22, 25],[22, 27, 14],[14, 24, 12],[22, 13, 24],[14, 15, 23],[24, 17, 11],[27, 28, 19],[20, 23, 26],[18, 16, 30],[13, 22, 27],[13, 27, 22],[13, 14, 11],[16, 13, 14],[19, 16, 20],[24, 15, 30],[23, 25, 26],[30, 25, 10],[18, 10, 26],[18, 20, 23],[24, 27, 20],[19, 26, 13],[15, 10, 12],[26, 19, 29],[21, 25, 16],[14, 17, 28],[29, 24, 18],[24, 26, 15],[21, 29, 28],[25, 17, 27],[29, 19, 18],[11, 13, 24],[30, 24, 21],[11, 15, 17],[20, 23, 17],[12, 28, 10],[20, 25, 16],[16, 11, 22],[30, 11, 15],[29, 15, 13],[25, 22, 14],[24, 30, 29],[15, 26, 17],[21, 27, 19],[18, 10, 14],[18, 20, 17],[29, 28, 15],[23, 29, 21],[11, 20, 22],[24, 12, 29],[10, 15, 16],[18, 27, 22],[26, 11, 29],[27, 24, 21],[30, 15, 24],[15, 26, 11],[30, 17, 13],[19, 27, 14],[23, 19, 30],[22, 18, 30],[24, 13, 12],[21, 25, 22],[13, 28, 23],[30, 23, 14],[25, 11, 23],[11, 29, 22],[15, 16, 13],[30, 11, 16],[28, 24, 17],[20, 16, 23],[23, 17, 24],[18, 15, 13],[27, 10, 29],[29, 19, 21],[14, 10, 30],[23, 29, 10],[30, 10, 15],[29, 20, 28],[17, 18, 12],[17, 22, 15],[19, 11, 13],[17, 27, 11],[21, 28, 24],[12, 11, 22],[20, 10, 30],[21, 16, 24],[22, 28, 30],[28, 12, 17],[23, 30, 21],[24, 21, 13],[30, 19, 23],[21, 29, 17],[24, 11, 16],[10, 28, 12],[17, 30, 19],[13, 17, 25],[23, 10, 24],[20, 12, 27],[26, 30, 11],[24, 12, 25],[27, 10, 17],[23, 18, 14],[20, 22, 11],[28, 19, 20],[25, 20, 12],[16, 14, 13],[23, 26, 14],[30, 15, 27],[12, 25, 22],[25, 28, 30],[28, 23, 17],[14, 18, 27],[12, 30, 14],[18, 21, 16],[15, 11, 10],[10, 22, 26],[26, 27, 10],[24, 18, 17],[24, 22, 20],[18, 30, 12],[17, 23, 25],[13, 16, 12],[18, 15, 14],[14, 28, 26],[18, 13, 22],[25, 10, 13],[17, 12, 29],[25, 23, 28],[13, 29, 19],[10, 11, 20],[21, 23, 17],[14, 29, 22],[19, 18, 12],[14, 15, 23],[18, 14, 19],[20, 15, 16],[26, 14, 19],[16, 25, 30],[15, 22, 21],[28, 23, 10],[26, 19, 28],[22, 17, 23],[15, 16, 19],[15, 26, 25],[29, 23, 18],[14, 30, 25],[25, 21, 24],[25, 11, 12],[20, 16, 22],[23, 30, 13],[26, 18, 11],[25, 23, 30],[15, 24, 10],[26, 21, 24],[23, 14, 30],[30, 24, 25],[24, 18, 21],[25, 11, 14],[12, 23, 26],[16, 13, 11],[22, 15, 18],[20, 28, 14],[22, 30, 29],[22, 30, 21],[13, 18, 20],[16, 23, 17],[27, 18, 29],[16, 27, 25],[21, 27, 15],[12, 22, 14],[17, 15, 19],[17, 27, 10],[20, 14, 10],[11, 26, 23],[12, 26, 24],[28, 30, 18],[17, 14, 18],[27, 29, 20],[24, 30, 13],[14, 25, 20],[25, 16, 11],[10, 21, 28],[25, 24, 23],[25, 12, 24],[25, 10, 22],[18, 14, 23],[13, 27, 10],[22, 29, 19],[27, 29, 23],[30, 19, 13],[17, 29, 23],[22, 11, 15],[24, 25, 14],[28, 19, 24],[16, 25, 28],[28, 16, 24],[23, 12, 27],[18, 26, 13],[21, 15, 12],[14, 19, 13],[30, 29, 12],[23, 30, 25],[24, 14, 28],[21, 30, 11],[24, 17, 18],[14, 25, 21],[26, 21, 20],[24, 12, 23],[20, 10, 15],[28, 18, 23],[18, 21, 25],[26, 13, 24],[30, 21, 24],[26, 13, 14],[17, 11, 18],[20, 26, 27],[10, 22, 18],[30, 23, 15],[13, 11, 30],[13, 28, 10],[19, 29, 10],[30, 15, 16],[21, 17, 19],[27, 29, 16],[30, 14, 23],[14, 27, 24],[19, 22, 17],[21, 23, 16],[27, 15, 12],[30, 20, 12],[21, 10, 28],[25, 11, 27],[21, 27, 13],[24, 17, 22],[13, 30, 29],[23, 13, 26],[23, 28, 29],[21, 17, 13],[29, 14, 15],[26, 30, 19],[13, 15, 24],[28, 16, 19],[28, 10, 24],[20, 13, 30],[14, 15, 20],[17, 16, 21],[15, 26, 27],[10, 21, 28],[15, 10, 21],[13, 12, 27],[28, 21, 23],[10, 11, 16],[11, 30, 29],[17, 21, 11],[14, 19, 29],[14, 18, 17],[22, 18, 15],[20, 28, 29],[17, 28, 19],[30, 25, 12],[22, 13, 21],[30, 25, 28],[13, 23, 25],[19, 11, 13],[21, 16, 23],[16, 21, 27],[26, 21, 29],[25, 28, 18],[11, 20, 26],[10, 23, 12],[17, 29, 26],[21, 17, 12],[24, 25, 11],[19, 15, 23],[18, 27, 24],[26, 14, 22],[17, 14, 21],[19, 24, 23],[20, 26, 25],[19, 23, 12],[19, 22, 24],[30, 28, 18],[14, 21, 19],[16, 14, 23],[20, 10, 26],[10, 20, 17],[16, 17, 26],[24, 17, 21],[16, 17, 10],[21, 30, 25],[20, 13, 26],[26, 29, 19],[14, 23, 25],[28, 20, 18],[27, 23, 30],[25, 17, 23],[22, 24, 21],[14, 17, 13],[24, 29, 18],[25, 26, 17],[18, 29, 24],[17, 20, 27],[29, 19, 16],[22, 18, 12],[23, 24, 15],[18, 16, 22],[20, 27, 19],[11, 25, 23],[14, 29, 23],[14, 30, 15],[18, 10, 26],[18, 22, 24],[27, 10, 22],[16, 20, 13],[10, 24, 18],[30, 14, 19],[18, 26, 16],[14, 19, 13],[25, 28, 19],[27, 30, 20],[17, 20, 21],[21, 13, 19],[25, 29, 28],[22, 28, 11],[23, 13, 28],[24, 16, 28],[17, 24, 16],[18, 16, 29],[26, 29, 16],[23, 15, 26],[12, 23, 11],[23, 10, 17],[22, 23, 18],[16, 30, 11],[23, 29, 14],[16, 29, 27],[11, 12, 22],[27, 28, 20],[26, 10, 20],[22, 30, 27],[26, 23, 18],[13, 26, 29],[18, 15, 25],[22, 27, 10],[24, 28, 25],[27, 22, 15],[15, 10, 11],[29, 21, 16],[24, 11, 16],[25, 11, 14],[23, 22, 10],[23, 27, 18],[27, 13, 12],[28, 30, 16],[26, 22, 10],[13, 12, 11],[25, 16, 28],[24, 23, 14],[11, 27, 12],[10, 20, 23],[18, 26, 23],[28, 26, 10],[16, 30, 21],[13, 18, 29],[15, 20, 16],[23, 25, 24],[19, 20, 21],[17, 22, 16],[13, 26, 28],[16, 24, 18],[17, 13, 28],[13, 10, 17],[27, 17, 22],[25, 12, 26],[11, 19, 24],[11, 19, 14],[24, 11, 21],[29, 27, 26],[25, 29, 10],[18, 25, 30],[22, 11, 18],[21, 14, 12],[17, 18, 19],[20, 17, 25],[11, 26, 15],[24, 23, 13],[17, 18, 16],[30, 22, 23],[23, 13, 25],[30, 27, 12],[25, 19, 17],[19, 23, 17],[24, 14, 19],[25, 18, 28],[26, 11, 16],[28, 24, 23],[25, 21, 10],[10, 29, 25],[22, 25, 26],[15, 26, 28],[26, 17, 12],[16, 18, 22],[10, 28, 18],[30, 16, 23],[14, 13, 25],[25, 13, 14],[16, 14, 19],[17, 16, 14],[30, 17, 26],[14, 19, 12],[11, 27, 24],[25, 17, 23],[26, 23, 12],[25, 29, 17],[10, 18, 26],[30, 16, 27],[13, 10, 29],[26, 28, 22],[21, 20, 14],[11, 25, 29],[28, 22, 19],[20, 29, 30],[24, 10, 30],[11, 10, 18],[10, 15, 22],[21, 13, 26],[23, 11, 20],[25, 16, 14],[11, 10, 30],[21, 28, 23],[13, 17, 14],[19, 23, 17],[13, 21, 24],[11, 27, 24],[27, 14, 16],[13, 15, 14],[18, 22, 14],[21, 12, 25],[14, 15, 12],[26, 25, 21],[25, 18, 11],[10, 20, 19],[23, 30, 20],[25, 26, 20],[12, 14, 10],[25, 20, 10],[30, 25, 11],[30, 26, 22],[21, 24, 15],[22, 12, 26],[21, 30, 28],[22, 11, 10],[16, 10, 29],[25, 17, 16],[10, 27, 22],[12, 22, 29],[11, 14, 24],[19, 28, 18],[14, 18, 20],[17, 26, 23],[29, 28, 22],[28, 21, 23],[27, 17, 14],[25, 22, 23],[18, 10, 29],[29, 21, 20],[16, 22, 23],[29, 11, 21],[25, 30, 26],[19, 10, 23],[20, 10, 11],[28, 26, 11],[13, 14, 12],[20, 15, 25],[26, 11, 17],[28, 19, 27],[13, 12, 23],[23, 27, 12],[30, 13, 29],[29, 23, 15],[12, 23, 18],[17, 21, 16],[24, 26, 11],[25, 11, 17],[15, 28, 13],[27, 13, 26],[19, 10, 22],[27, 28, 15],[10, 17, 28],[20, 28, 27],[21, 24, 20],[16, 26, 17],[13, 17, 11],[27, 17, 16],[12, 20, 10],[20, 30, 18],[20, 19, 13],[29, 28, 15],[26, 20, 19],[20, 24, 22],[23, 25, 20],[11, 18, 16],[29, 20, 30],[22, 17, 21],[14, 20, 22],[29, 20, 25],[30, 19, 13],[25, 27, 16],[15, 26, 29],[21, 23, 29],[19, 26, 24],[11, 15, 10],[29, 15, 12],[13, 29, 20],[12, 16, 21],[17, 19, 20],[25, 16, 26],[29, 28, 23],[19, 13, 12],[24, 16, 14],[26, 24, 12],[15, 18, 30],[29, 20, 11],[18, 17, 22],[12, 11, 22],[18, 30, 20],[22, 11, 10],[13, 16, 21],[18, 27, 12],[11, 16, 10],[24, 19, 22],[13, 24, 19],[30, 21, 24],[13, 29, 18],[30, 27, 12],[16, 30, 28],[29, 13, 27],[29, 11, 27],[21, 30, 27],[30, 22, 18],[30, 23, 20],[15, 14, 21],[21, 20, 10],[10, 20, 25],[10, 15, 12],[16, 12, 18],[12, 21, 26],[28, 22, 27],[13, 27, 14],[17, 26, 16],[24, 15, 28],[19, 27, 26],[13, 15, 25],[16, 26, 22],[23, 24, 18],[24, 27, 30],[20, 24, 22],[22, 30, 27],[12, 10, 17],[14, 30, 13],[16, 13, 26],[21, 30, 13],[28, 17, 10],[15, 18, 28],[24, 29, 25],[17, 16, 14],[13, 16, 22],[15, 20, 22],[28, 20, 29],[15, 14, 24],[16, 11, 19],[20, 29, 17],[12, 20, 14],[18, 22, 11],[19, 28, 17],[26, 29, 11],[20, 25, 18],[24, 28, 14],[11, 25, 19],[11, 29, 20],[28, 12, 21],[26, 12, 21],[28, 14, 26],[19, 23, 14],[29, 13, 16],[12, 17, 11],[15, 21, 27],[17, 13, 29],[18, 28, 29],[22, 29, 26],[13, 26, 28],[29, 14, 30],[17, 20, 24],[27, 29, 20],[28, 30, 20],[13, 20, 28],[11, 29, 16],[18, 16, 19],[26, 19, 14],[22, 12, 19],[23, 26, 30],[15, 21, 30],[12, 26, 27],[16, 27, 12],[20, 15, 25],[16, 10, 22],[25, 21, 27],[23, 24, 12],[16, 10, 27],[17, 25, 12],[11, 18, 14],[17, 14, 29],[24, 30, 23],[11, 17, 24],[16, 29, 10],[16, 13, 17],[15, 10, 16],[24, 26, 19],[27, 24, 21],[26, 15, 22],[30, 14, 10],[30, 11, 27],[29, 22, 17],[25, 15, 26],[29, 19, 11],[13, 22, 23],[22, 12, 17],[12, 17, 19],[27, 20, 15],[18, 21, 22],[16, 24, 18],[19, 30, 20],[30, 26, 23],[26, 30, 21],[12, 16, 22],[15, 22, 11],[14, 10, 13],[15, 26, 20],[25, 12, 30],[21, 11, 24],[30, 20, 19],[26, 11, 18],[13, 21, 11],[27, 18, 21],[21, 29, 13],[18, 29, 21],[23, 24, 20],[28, 11, 21],[26, 28, 19],[12, 28, 16],[12, 26, 22],[19, 17, 15],[16, 26, 10],[11, 15, 20],[24, 11, 29],[20, 16, 19],[20, 23, 13],[16, 25, 11],[23, 21, 26],[22, 27, 24],[19, 18, 14],[24, 27, 18],[10, 15, 13],[14, 17, 29],[19, 14, 13],[20, 11, 22],[17, 19, 14],[19, 18, 16],[15, 24, 18],[12, 27, 21],[21, 13, 11],[20, 17, 16],[19, 22, 11],[25, 30, 16],[10, 29, 30],[19, 21, 28],[19, 13, 27],[12, 14, 27],[17, 10, 22],[10, 22, 15],[14, 26, 30],[17, 25, 28],[12, 24, 18],[26, 27, 18],[12, 20, 16],[16, 22, 20],[19, 15, 23],[16, 30, 27],[26, 27, 30],[24, 18, 10],[22, 25, 14],[12, 27, 30],[21, 24, 26],[15, 19, 10],[18, 25, 14],[11, 14, 28],[23, 22, 18],[17, 24, 14],[28, 16, 25],[16, 12, 26],[26, 15, 11],[11, 22, 26],[21, 10, 23],[30, 16, 17],[12, 13, 16],[22, 18, 24],[26, 20, 10],[20, 25, 17],[24, 22, 12],[26, 24, 16],[17, 22, 26],[13, 10, 14],[13, 24, 26],[27, 13, 28],[10, 26, 20],[15, 18, 17],[15, 25, 18],[30, 17, 25],[25, 29, 26],[17, 27, 28],[21, 30, 19],[16, 21, 25],[16, 15, 25],[23, 20, 27],[27, 12, 15],[19, 30, 23],[15, 19, 10],[25, 21, 30],[16, 13, 19],[22, 25, 27],[28, 14, 10],[10, 29, 28],[20, 23, 12],[15, 17, 30],[29, 11, 24],[11, 22, 19],[16, 17, 11],[27, 18, 23],[22, 19, 10],[25, 30, 19],[12, 29, 25],[16, 17, 11],[14, 23, 24],[29, 26, 20],[14, 15, 24],[26, 16, 21],[13, 10, 23],[13, 16, 24],[26, 16, 28],[14, 19, 13],[17, 16, 22],[14, 17, 20],[23, 12, 21],[22, 18, 17],[10, 16, 11],[19, 26, 18],[21, 12, 23],[28, 23, 18],[29, 12, 10],[22, 27, 14],[14, 20, 16],[14, 12, 20],[10, 13, 17],[16, 19, 30],[26, 11, 21],[11, 27, 28],[26, 10, 27],[15, 18, 16],[26, 19, 12],[21, 27, 14],[21, 30, 13],[21, 23, 13],[18, 15, 20],[24, 14, 12],[12, 26, 15],[26, 30, 13],[26, 13, 25],[18, 28, 10],[22, 29, 17],[25, 28, 16],[27, 12, 15],[18, 10, 16],[29, 23, 22],[25, 13, 27],[29, 14, 26],[30, 23, 19],[12, 21, 17],[30, 24, 15],[13, 21, 25],[29, 18, 16],[26, 20, 28],[13, 24, 29],[28, 23, 25],[24, 21, 27],[17, 13, 27],[28, 23, 30],[12, 19, 25],[26, 16, 29],[25, 24, 29],[21, 19, 10],[15, 25, 28],[14, 24, 15],[23, 14, 18],[15, 11, 17],[16, 21, 14],[21, 24, 17],[21, 23, 14],[25, 20, 16],[26, 11, 14],[12, 24, 23],[22, 21, 19],[20, 18, 25],[29, 25, 24],[19, 23, 20],[17, 30, 10],[22, 13, 15],[27, 28, 30],[15, 16, 29],[21, 13, 22],[18, 19, 21],[11, 12, 30],[29, 25, 13],[23, 13, 24],[12, 14, 17],[20, 16, 26],[15, 13, 30],[12, 26, 24],[18, 14, 24],[21, 23, 10],[29, 17, 18],[15, 22, 16],[23, 17, 26],[30, 20, 17],[13, 30, 22],[16, 15, 24],[30, 27, 28],[22, 12, 21],[22, 11, 14],[20, 29, 19],[30, 28, 19],[24, 20, 17],[13, 18, 26],[30, 23, 16],[18, 16, 22]]}

MUTATIONS = 2
NUM_ITEMS = len(knapsack_data["items"])
BACKPACK_SIZE, BACKPACK_WEIGHT = knapsack_data["max_size"], knapsack_data["max_weight"]
ITEMS = knapsack_data["items"]

ga = GA(fitness, generate_random_individual, create_child, clones=1, parents=2, population_size=100)
             
GENERATION_LOG = Template('[$time] - Generation $generation - Average knapsack value of $average_knapsack_value')
BEST_STATISTICS = Template('Fittest with $fitness knapsack value')
for generation in range(20):
    ga.next()
    knapsack_values = list(map(lambda x: get_knapsack_value(x), ga.population))
    average_knapsack_value = sum(knapsack_values) / float(len(knapsack_values))
    fittest_knapsack_value = get_knapsack_value(ga.fittest_individual)
    print(GENERATION_LOG.substitute(time=datetime.datetime.now(),generation=generation, average_knapsack_value=average_knapsack_value))
    print(BEST_STATISTICS.substitute(fitness=fittest_knapsack_value))

In [ ]:
# ASSIGNMENT 3
#
# We can see that the average fitness goes up slowly, but also oscilates quite noticably. It seems that there
# is too much randomness in the creation of a new generation which makes the convergence slow. See for example
# how the parents are selected for creating children (GA._choose_parents()). The parents are selected based
# randomly weight by their fitness. But does this favor 'good' parents enough?
# 
# Try to find adjustments to the fitness function that result in a better result
import math
random.seed(1)

def fitness(individual):
    get_knapsack_value(individual)

# You can also fiddle with:
#  - clones (amount of best ranked individuals propagated to the next generation)
#  - parents (hmmm...)
#  - population_size
#
# And if you are really fealing adventurous, try to write a better create_child function to influence the 
# next generation
def create_child(parents):
    child = crossover(parents)
    for _ in range(MUTATIONS):
        mutate(child)
    return child

knapsack_data = {
    'max_size':200,        # the maximum allowed total size of items in the knapsack
    'max_weight':200,      # the maximum allowed total weight of items in the knapsack
    'items':[[29, 18, 25], # the item descriptions [size, weight, value]
             [10, 26, 21],[10, 29, 26],[24, 11, 16],[12, 19, 25],[20, 19, 12],[14, 11, 12],[17, 24, 28],[21, 19, 26],[12, 18, 26],[27, 30, 18],[23, 26, 20],[26, 30, 18],[19, 13, 30],[27, 11, 30],[19, 23, 17],[18, 27, 29],[27, 21, 15],[22, 14, 19],[14, 26, 22],[24, 10, 28],[20, 29, 12],[11, 14, 15],[17, 24, 15],[29, 24, 14],[26, 16, 23],[27, 15, 29],[11, 27, 18],[11, 10, 28],[25, 22, 14],[11, 16, 26],[14, 23, 26],[23, 18, 17],[13, 27, 28],[18, 21, 15],[23, 14, 10],[18, 25, 29],[27, 20, 21],[23, 12, 21],[20, 18, 26],[11, 23, 10],[13, 23, 29],[23, 16, 13],[16, 28, 18],[15, 20, 13],[12, 27, 22],[24, 22, 15],[30, 24, 14],[19, 16, 30],[22, 18, 21],[11, 28, 30],[28, 24, 17],[18, 29, 22],[23, 11, 10],[17, 11, 24],[14, 27, 18],[11, 18, 14],[12, 29, 26],[17, 24, 23],[17, 25, 14],[10, 12, 15],[14, 23, 25],[12, 25, 15],[11, 18, 26],[29, 24, 21],[29, 20, 24],[11, 10, 24],[24, 12, 17],[13, 26, 22],[17, 12, 15],[10, 24, 30],[10, 25, 14],[25, 18, 26],[23, 21, 19],[25, 28, 21],[10, 22, 11],[11, 23, 29],[13, 12, 29],[30, 25, 26],[18, 15, 23],[10, 25, 16],[14, 19, 10],[17, 12, 24],[27, 19, 29],[28, 22, 18],[10, 23, 30],[24, 21, 13],[20, 23, 24],[28, 17, 15],[12, 21, 11],[12, 10, 17],[23, 13, 15],[13, 22, 27],[11, 18, 15],[18, 28, 17],[23, 20, 28],[28, 30, 27],[23, 13, 12],[13, 23, 14],[12, 21, 24],[10, 15, 30],[25, 30, 13],[19, 23, 11],[26, 11, 16],[15, 30, 16],[14, 21, 16],[18, 16, 29],[10, 30, 16],[19, 28, 10],[15, 17, 12],[18, 28, 13],[26, 24, 19],[16, 14, 25],[15, 11, 10],[24, 18, 13],[27, 14, 15],[16, 15, 21],[14, 26, 19],[11, 21, 22],[25, 15, 23],[12, 27, 29],[19, 22, 24],[23, 29, 18],[26, 13, 24],[10, 14, 29],[28, 21, 29],[22, 24, 28],[20, 18, 15],[26, 28, 22],[29, 18, 12],[13, 26, 28],[26, 21, 12],[28, 17, 26],[23, 14, 21],[10, 29, 12],[18, 26, 15],[15, 22, 23],[22, 29, 17],[17, 10, 26],[12, 19, 26],[27, 15, 12],[12, 11, 26],[14, 29, 28],[26, 11, 13],[29, 14, 19],[20, 26, 25],[29, 15, 12],[14, 11, 12],[21, 19, 22],[27, 17, 24],[14, 15, 30],[23, 14, 21],[21, 28, 12],[10, 19, 20],[28, 10, 29],[26, 21, 24],[18, 26, 11],[27, 22, 25],[22, 27, 14],[14, 24, 12],[22, 13, 24],[14, 15, 23],[24, 17, 11],[27, 28, 19],[20, 23, 26],[18, 16, 30],[13, 22, 27],[13, 27, 22],[13, 14, 11],[16, 13, 14],[19, 16, 20],[24, 15, 30],[23, 25, 26],[30, 25, 10],[18, 10, 26],[18, 20, 23],[24, 27, 20],[19, 26, 13],[15, 10, 12],[26, 19, 29],[21, 25, 16],[14, 17, 28],[29, 24, 18],[24, 26, 15],[21, 29, 28],[25, 17, 27],[29, 19, 18],[11, 13, 24],[30, 24, 21],[11, 15, 17],[20, 23, 17],[12, 28, 10],[20, 25, 16],[16, 11, 22],[30, 11, 15],[29, 15, 13],[25, 22, 14],[24, 30, 29],[15, 26, 17],[21, 27, 19],[18, 10, 14],[18, 20, 17],[29, 28, 15],[23, 29, 21],[11, 20, 22],[24, 12, 29],[10, 15, 16],[18, 27, 22],[26, 11, 29],[27, 24, 21],[30, 15, 24],[15, 26, 11],[30, 17, 13],[19, 27, 14],[23, 19, 30],[22, 18, 30],[24, 13, 12],[21, 25, 22],[13, 28, 23],[30, 23, 14],[25, 11, 23],[11, 29, 22],[15, 16, 13],[30, 11, 16],[28, 24, 17],[20, 16, 23],[23, 17, 24],[18, 15, 13],[27, 10, 29],[29, 19, 21],[14, 10, 30],[23, 29, 10],[30, 10, 15],[29, 20, 28],[17, 18, 12],[17, 22, 15],[19, 11, 13],[17, 27, 11],[21, 28, 24],[12, 11, 22],[20, 10, 30],[21, 16, 24],[22, 28, 30],[28, 12, 17],[23, 30, 21],[24, 21, 13],[30, 19, 23],[21, 29, 17],[24, 11, 16],[10, 28, 12],[17, 30, 19],[13, 17, 25],[23, 10, 24],[20, 12, 27],[26, 30, 11],[24, 12, 25],[27, 10, 17],[23, 18, 14],[20, 22, 11],[28, 19, 20],[25, 20, 12],[16, 14, 13],[23, 26, 14],[30, 15, 27],[12, 25, 22],[25, 28, 30],[28, 23, 17],[14, 18, 27],[12, 30, 14],[18, 21, 16],[15, 11, 10],[10, 22, 26],[26, 27, 10],[24, 18, 17],[24, 22, 20],[18, 30, 12],[17, 23, 25],[13, 16, 12],[18, 15, 14],[14, 28, 26],[18, 13, 22],[25, 10, 13],[17, 12, 29],[25, 23, 28],[13, 29, 19],[10, 11, 20],[21, 23, 17],[14, 29, 22],[19, 18, 12],[14, 15, 23],[18, 14, 19],[20, 15, 16],[26, 14, 19],[16, 25, 30],[15, 22, 21],[28, 23, 10],[26, 19, 28],[22, 17, 23],[15, 16, 19],[15, 26, 25],[29, 23, 18],[14, 30, 25],[25, 21, 24],[25, 11, 12],[20, 16, 22],[23, 30, 13],[26, 18, 11],[25, 23, 30],[15, 24, 10],[26, 21, 24],[23, 14, 30],[30, 24, 25],[24, 18, 21],[25, 11, 14],[12, 23, 26],[16, 13, 11],[22, 15, 18],[20, 28, 14],[22, 30, 29],[22, 30, 21],[13, 18, 20],[16, 23, 17],[27, 18, 29],[16, 27, 25],[21, 27, 15],[12, 22, 14],[17, 15, 19],[17, 27, 10],[20, 14, 10],[11, 26, 23],[12, 26, 24],[28, 30, 18],[17, 14, 18],[27, 29, 20],[24, 30, 13],[14, 25, 20],[25, 16, 11],[10, 21, 28],[25, 24, 23],[25, 12, 24],[25, 10, 22],[18, 14, 23],[13, 27, 10],[22, 29, 19],[27, 29, 23],[30, 19, 13],[17, 29, 23],[22, 11, 15],[24, 25, 14],[28, 19, 24],[16, 25, 28],[28, 16, 24],[23, 12, 27],[18, 26, 13],[21, 15, 12],[14, 19, 13],[30, 29, 12],[23, 30, 25],[24, 14, 28],[21, 30, 11],[24, 17, 18],[14, 25, 21],[26, 21, 20],[24, 12, 23],[20, 10, 15],[28, 18, 23],[18, 21, 25],[26, 13, 24],[30, 21, 24],[26, 13, 14],[17, 11, 18],[20, 26, 27],[10, 22, 18],[30, 23, 15],[13, 11, 30],[13, 28, 10],[19, 29, 10],[30, 15, 16],[21, 17, 19],[27, 29, 16],[30, 14, 23],[14, 27, 24],[19, 22, 17],[21, 23, 16],[27, 15, 12],[30, 20, 12],[21, 10, 28],[25, 11, 27],[21, 27, 13],[24, 17, 22],[13, 30, 29],[23, 13, 26],[23, 28, 29],[21, 17, 13],[29, 14, 15],[26, 30, 19],[13, 15, 24],[28, 16, 19],[28, 10, 24],[20, 13, 30],[14, 15, 20],[17, 16, 21],[15, 26, 27],[10, 21, 28],[15, 10, 21],[13, 12, 27],[28, 21, 23],[10, 11, 16],[11, 30, 29],[17, 21, 11],[14, 19, 29],[14, 18, 17],[22, 18, 15],[20, 28, 29],[17, 28, 19],[30, 25, 12],[22, 13, 21],[30, 25, 28],[13, 23, 25],[19, 11, 13],[21, 16, 23],[16, 21, 27],[26, 21, 29],[25, 28, 18],[11, 20, 26],[10, 23, 12],[17, 29, 26],[21, 17, 12],[24, 25, 11],[19, 15, 23],[18, 27, 24],[26, 14, 22],[17, 14, 21],[19, 24, 23],[20, 26, 25],[19, 23, 12],[19, 22, 24],[30, 28, 18],[14, 21, 19],[16, 14, 23],[20, 10, 26],[10, 20, 17],[16, 17, 26],[24, 17, 21],[16, 17, 10],[21, 30, 25],[20, 13, 26],[26, 29, 19],[14, 23, 25],[28, 20, 18],[27, 23, 30],[25, 17, 23],[22, 24, 21],[14, 17, 13],[24, 29, 18],[25, 26, 17],[18, 29, 24],[17, 20, 27],[29, 19, 16],[22, 18, 12],[23, 24, 15],[18, 16, 22],[20, 27, 19],[11, 25, 23],[14, 29, 23],[14, 30, 15],[18, 10, 26],[18, 22, 24],[27, 10, 22],[16, 20, 13],[10, 24, 18],[30, 14, 19],[18, 26, 16],[14, 19, 13],[25, 28, 19],[27, 30, 20],[17, 20, 21],[21, 13, 19],[25, 29, 28],[22, 28, 11],[23, 13, 28],[24, 16, 28],[17, 24, 16],[18, 16, 29],[26, 29, 16],[23, 15, 26],[12, 23, 11],[23, 10, 17],[22, 23, 18],[16, 30, 11],[23, 29, 14],[16, 29, 27],[11, 12, 22],[27, 28, 20],[26, 10, 20],[22, 30, 27],[26, 23, 18],[13, 26, 29],[18, 15, 25],[22, 27, 10],[24, 28, 25],[27, 22, 15],[15, 10, 11],[29, 21, 16],[24, 11, 16],[25, 11, 14],[23, 22, 10],[23, 27, 18],[27, 13, 12],[28, 30, 16],[26, 22, 10],[13, 12, 11],[25, 16, 28],[24, 23, 14],[11, 27, 12],[10, 20, 23],[18, 26, 23],[28, 26, 10],[16, 30, 21],[13, 18, 29],[15, 20, 16],[23, 25, 24],[19, 20, 21],[17, 22, 16],[13, 26, 28],[16, 24, 18],[17, 13, 28],[13, 10, 17],[27, 17, 22],[25, 12, 26],[11, 19, 24],[11, 19, 14],[24, 11, 21],[29, 27, 26],[25, 29, 10],[18, 25, 30],[22, 11, 18],[21, 14, 12],[17, 18, 19],[20, 17, 25],[11, 26, 15],[24, 23, 13],[17, 18, 16],[30, 22, 23],[23, 13, 25],[30, 27, 12],[25, 19, 17],[19, 23, 17],[24, 14, 19],[25, 18, 28],[26, 11, 16],[28, 24, 23],[25, 21, 10],[10, 29, 25],[22, 25, 26],[15, 26, 28],[26, 17, 12],[16, 18, 22],[10, 28, 18],[30, 16, 23],[14, 13, 25],[25, 13, 14],[16, 14, 19],[17, 16, 14],[30, 17, 26],[14, 19, 12],[11, 27, 24],[25, 17, 23],[26, 23, 12],[25, 29, 17],[10, 18, 26],[30, 16, 27],[13, 10, 29],[26, 28, 22],[21, 20, 14],[11, 25, 29],[28, 22, 19],[20, 29, 30],[24, 10, 30],[11, 10, 18],[10, 15, 22],[21, 13, 26],[23, 11, 20],[25, 16, 14],[11, 10, 30],[21, 28, 23],[13, 17, 14],[19, 23, 17],[13, 21, 24],[11, 27, 24],[27, 14, 16],[13, 15, 14],[18, 22, 14],[21, 12, 25],[14, 15, 12],[26, 25, 21],[25, 18, 11],[10, 20, 19],[23, 30, 20],[25, 26, 20],[12, 14, 10],[25, 20, 10],[30, 25, 11],[30, 26, 22],[21, 24, 15],[22, 12, 26],[21, 30, 28],[22, 11, 10],[16, 10, 29],[25, 17, 16],[10, 27, 22],[12, 22, 29],[11, 14, 24],[19, 28, 18],[14, 18, 20],[17, 26, 23],[29, 28, 22],[28, 21, 23],[27, 17, 14],[25, 22, 23],[18, 10, 29],[29, 21, 20],[16, 22, 23],[29, 11, 21],[25, 30, 26],[19, 10, 23],[20, 10, 11],[28, 26, 11],[13, 14, 12],[20, 15, 25],[26, 11, 17],[28, 19, 27],[13, 12, 23],[23, 27, 12],[30, 13, 29],[29, 23, 15],[12, 23, 18],[17, 21, 16],[24, 26, 11],[25, 11, 17],[15, 28, 13],[27, 13, 26],[19, 10, 22],[27, 28, 15],[10, 17, 28],[20, 28, 27],[21, 24, 20],[16, 26, 17],[13, 17, 11],[27, 17, 16],[12, 20, 10],[20, 30, 18],[20, 19, 13],[29, 28, 15],[26, 20, 19],[20, 24, 22],[23, 25, 20],[11, 18, 16],[29, 20, 30],[22, 17, 21],[14, 20, 22],[29, 20, 25],[30, 19, 13],[25, 27, 16],[15, 26, 29],[21, 23, 29],[19, 26, 24],[11, 15, 10],[29, 15, 12],[13, 29, 20],[12, 16, 21],[17, 19, 20],[25, 16, 26],[29, 28, 23],[19, 13, 12],[24, 16, 14],[26, 24, 12],[15, 18, 30],[29, 20, 11],[18, 17, 22],[12, 11, 22],[18, 30, 20],[22, 11, 10],[13, 16, 21],[18, 27, 12],[11, 16, 10],[24, 19, 22],[13, 24, 19],[30, 21, 24],[13, 29, 18],[30, 27, 12],[16, 30, 28],[29, 13, 27],[29, 11, 27],[21, 30, 27],[30, 22, 18],[30, 23, 20],[15, 14, 21],[21, 20, 10],[10, 20, 25],[10, 15, 12],[16, 12, 18],[12, 21, 26],[28, 22, 27],[13, 27, 14],[17, 26, 16],[24, 15, 28],[19, 27, 26],[13, 15, 25],[16, 26, 22],[23, 24, 18],[24, 27, 30],[20, 24, 22],[22, 30, 27],[12, 10, 17],[14, 30, 13],[16, 13, 26],[21, 30, 13],[28, 17, 10],[15, 18, 28],[24, 29, 25],[17, 16, 14],[13, 16, 22],[15, 20, 22],[28, 20, 29],[15, 14, 24],[16, 11, 19],[20, 29, 17],[12, 20, 14],[18, 22, 11],[19, 28, 17],[26, 29, 11],[20, 25, 18],[24, 28, 14],[11, 25, 19],[11, 29, 20],[28, 12, 21],[26, 12, 21],[28, 14, 26],[19, 23, 14],[29, 13, 16],[12, 17, 11],[15, 21, 27],[17, 13, 29],[18, 28, 29],[22, 29, 26],[13, 26, 28],[29, 14, 30],[17, 20, 24],[27, 29, 20],[28, 30, 20],[13, 20, 28],[11, 29, 16],[18, 16, 19],[26, 19, 14],[22, 12, 19],[23, 26, 30],[15, 21, 30],[12, 26, 27],[16, 27, 12],[20, 15, 25],[16, 10, 22],[25, 21, 27],[23, 24, 12],[16, 10, 27],[17, 25, 12],[11, 18, 14],[17, 14, 29],[24, 30, 23],[11, 17, 24],[16, 29, 10],[16, 13, 17],[15, 10, 16],[24, 26, 19],[27, 24, 21],[26, 15, 22],[30, 14, 10],[30, 11, 27],[29, 22, 17],[25, 15, 26],[29, 19, 11],[13, 22, 23],[22, 12, 17],[12, 17, 19],[27, 20, 15],[18, 21, 22],[16, 24, 18],[19, 30, 20],[30, 26, 23],[26, 30, 21],[12, 16, 22],[15, 22, 11],[14, 10, 13],[15, 26, 20],[25, 12, 30],[21, 11, 24],[30, 20, 19],[26, 11, 18],[13, 21, 11],[27, 18, 21],[21, 29, 13],[18, 29, 21],[23, 24, 20],[28, 11, 21],[26, 28, 19],[12, 28, 16],[12, 26, 22],[19, 17, 15],[16, 26, 10],[11, 15, 20],[24, 11, 29],[20, 16, 19],[20, 23, 13],[16, 25, 11],[23, 21, 26],[22, 27, 24],[19, 18, 14],[24, 27, 18],[10, 15, 13],[14, 17, 29],[19, 14, 13],[20, 11, 22],[17, 19, 14],[19, 18, 16],[15, 24, 18],[12, 27, 21],[21, 13, 11],[20, 17, 16],[19, 22, 11],[25, 30, 16],[10, 29, 30],[19, 21, 28],[19, 13, 27],[12, 14, 27],[17, 10, 22],[10, 22, 15],[14, 26, 30],[17, 25, 28],[12, 24, 18],[26, 27, 18],[12, 20, 16],[16, 22, 20],[19, 15, 23],[16, 30, 27],[26, 27, 30],[24, 18, 10],[22, 25, 14],[12, 27, 30],[21, 24, 26],[15, 19, 10],[18, 25, 14],[11, 14, 28],[23, 22, 18],[17, 24, 14],[28, 16, 25],[16, 12, 26],[26, 15, 11],[11, 22, 26],[21, 10, 23],[30, 16, 17],[12, 13, 16],[22, 18, 24],[26, 20, 10],[20, 25, 17],[24, 22, 12],[26, 24, 16],[17, 22, 26],[13, 10, 14],[13, 24, 26],[27, 13, 28],[10, 26, 20],[15, 18, 17],[15, 25, 18],[30, 17, 25],[25, 29, 26],[17, 27, 28],[21, 30, 19],[16, 21, 25],[16, 15, 25],[23, 20, 27],[27, 12, 15],[19, 30, 23],[15, 19, 10],[25, 21, 30],[16, 13, 19],[22, 25, 27],[28, 14, 10],[10, 29, 28],[20, 23, 12],[15, 17, 30],[29, 11, 24],[11, 22, 19],[16, 17, 11],[27, 18, 23],[22, 19, 10],[25, 30, 19],[12, 29, 25],[16, 17, 11],[14, 23, 24],[29, 26, 20],[14, 15, 24],[26, 16, 21],[13, 10, 23],[13, 16, 24],[26, 16, 28],[14, 19, 13],[17, 16, 22],[14, 17, 20],[23, 12, 21],[22, 18, 17],[10, 16, 11],[19, 26, 18],[21, 12, 23],[28, 23, 18],[29, 12, 10],[22, 27, 14],[14, 20, 16],[14, 12, 20],[10, 13, 17],[16, 19, 30],[26, 11, 21],[11, 27, 28],[26, 10, 27],[15, 18, 16],[26, 19, 12],[21, 27, 14],[21, 30, 13],[21, 23, 13],[18, 15, 20],[24, 14, 12],[12, 26, 15],[26, 30, 13],[26, 13, 25],[18, 28, 10],[22, 29, 17],[25, 28, 16],[27, 12, 15],[18, 10, 16],[29, 23, 22],[25, 13, 27],[29, 14, 26],[30, 23, 19],[12, 21, 17],[30, 24, 15],[13, 21, 25],[29, 18, 16],[26, 20, 28],[13, 24, 29],[28, 23, 25],[24, 21, 27],[17, 13, 27],[28, 23, 30],[12, 19, 25],[26, 16, 29],[25, 24, 29],[21, 19, 10],[15, 25, 28],[14, 24, 15],[23, 14, 18],[15, 11, 17],[16, 21, 14],[21, 24, 17],[21, 23, 14],[25, 20, 16],[26, 11, 14],[12, 24, 23],[22, 21, 19],[20, 18, 25],[29, 25, 24],[19, 23, 20],[17, 30, 10],[22, 13, 15],[27, 28, 30],[15, 16, 29],[21, 13, 22],[18, 19, 21],[11, 12, 30],[29, 25, 13],[23, 13, 24],[12, 14, 17],[20, 16, 26],[15, 13, 30],[12, 26, 24],[18, 14, 24],[21, 23, 10],[29, 17, 18],[15, 22, 16],[23, 17, 26],[30, 20, 17],[13, 30, 22],[16, 15, 24],[30, 27, 28],[22, 12, 21],[22, 11, 14],[20, 29, 19],[30, 28, 19],[24, 20, 17],[13, 18, 26],[30, 23, 16],[18, 16, 22]]}

MUTATIONS = 2
NUM_ITEMS = len(knapsack_data["items"])
BACKPACK_SIZE, BACKPACK_WEIGHT = knapsack_data["max_size"], knapsack_data["max_weight"]
ITEMS = knapsack_data["items"]

ga = GA(fitness, generate_random_individual, create_child, clones=1, parents=2, population_size=100)
             
GENERATION_LOG = Template('[$time] - Generation $generation - Average knapsack value of $average_knapsack_value')
BEST_STATISTICS = Template('Fittest with $fitness knapsack value')
for generation in range(20):
    ga.next()
    knapsack_values = list(map(lambda x: get_knapsack_value(x), ga.population))
    average_knapsack_value = sum(knapsack_values) / float(len(knapsack_values))
    fittest_knapsack_value = get_knapsack_value(ga.fittest_individual)
    print(GENERATION_LOG.substitute(time=datetime.datetime.now(),generation=generation, average_knapsack_value=average_knapsack_value))
    print(BEST_STATISTICS.substitute(fitness=fittest_knapsack_value))